04월 05일 목요일 : 
===
비가 부슬부슬 내린다. 파이썬은 봄비와도 같다.

# Modules and Package
---
모듈 : 함수나 변수 또는 클래스들을 모아 놓은 파일. py 확장자를 갖는 일반적인 하나의 파이썬 파일
    * 여러 모듈에 나누어 작성하는 것이 관리에 이점이 있다.




In [1]:
import sys
for path in sys.path:
    print(path)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/extensions
/Users/KangSooSang/.ipython


# 배틀그라운드를 간단하게 만들어 보자!
---
## 게임 개요
* 100 명의 플레이어가 맵의 다양한 위치에 떨어진다.
* 지도 상의 안전 지역으로 표시된 흰색 서클 안으로 이동하지 않으면 지속적으로 데미지를 입는다.
* 안전 지역은 점점 좁아지며, 이동 중에 마주친 플레이어들끼리 서로 전투를 하여 생존자 수가 줄어든다.
* 최후의 생존자가 승리하게 된다.

## event.py
---
* random.py 모듈의 randint 함수를 import
* MAX_LOCATION = 100 상수 정의

** Function : get_random_location **
0. 한 플레이어의 위치를 임의로 정해준다.
1. parameter
    * 없음
2. return
    * 0 ~ MAX_LOCATION 사이의 정수 값  

In [41]:
from random import randint

MAX_LOCATION = 100

def get_random_location():
    return randint(0,MAX_LOCATION)

def battle(player_list):
    print("battles between players!")
    winner_index = randint(0, len(player_list)-1)
    for i in range(len(player_list)):
        if i == winner_index:
            player_list[i].health += 10*(len(player_list)-1)
            print("\t Player %d wins!" %player_list[i].id)
        else:
            player_list[i].health = 0
            print("\t Player %d loses..." %player_list[i].id)

if __name__ == '__main__':
    for i in range(10):
        print(get_random_location())

64
17
5
23
11
11
17
50
58
51


## player.py
---
** Class : Player **
0. 플레이어가 가지는 속성을 정의해준다.
1. attributes
    * id
        * 임의의 정수형 숫자
        * 생성자 함수로 입력받음
    * status
        * 'alive'로 초기화
    * health
        * 100으로 초기화된 체력
    * location
        * 1차원 정수 좌표를 나타내는 리스트
        * event.py 모듈 내 get_random_location() 함수를 호출하여 초기화
2. methods
    * `__str__`(self)
        * 객체의 attributes 를 출력 가능하도록 오버라이드
        * 플레이어가 살아있는지에 따라 다르게 출력
    * run(self, target_location)
        * 현재 객체의 위치가 target 위치와 같으면 객체의 위치를 변경하지 않음
        * 위치가 다르면, target 위치 방향으로 한 칸을 움직임
        * self.location 을 업데이트하여 위치를 변경
    * status_update(self)
        * health 가 0이 되면 status attribute 를 'dead'로 변경

In [36]:
# from event import get_random_location

class Player:
    def __init__(self, id):
        self.id = id
        self.status = 'alive'
        self.health = 100
        self.location = get_random_location()
        
    def __str__(self):
        player_inform = []
        player_inform.append('Player %d is %s.' %(self.id, self.status))
        player_inform.append('health = %d' %self.health)
        player_inform.append('location = %d' %self.location)
        
        player_inform = ' '.join(player_inform)
        return player_inform
    
    def run(self, target_location):
        if self.location == target_location:
            return
        else:
            movement = 1*(self.location < target_location) - 1*(self.location > target_location)
            self.location += movement
    
    def status_update(self):
        if self.health == 0:
            self.status = 'dead'
    

if __name__ == '__main__':
    p = Player(1)
    for i in range(10):
        p.run(10)
        print(p)

Player 1 is alive. health = 100 location = 89
Player 1 is alive. health = 100 location = 88
Player 1 is alive. health = 100 location = 87
Player 1 is alive. health = 100 location = 86
Player 1 is alive. health = 100 location = 85
Player 1 is alive. health = 100 location = 84
Player 1 is alive. health = 100 location = 83
Player 1 is alive. health = 100 location = 82
Player 1 is alive. health = 100 location = 81
Player 1 is alive. health = 100 location = 80


## map.py
---
** Function : get_safe_zone **
0. 경과한 시간에 따라 target 위치로부터 얼마만큼 떨어진 거리까지가 safe zone 인지를 나타낸다.
1. parameter
    * 현재 시간, 파라미터 time 은 게임 시작 후 경과한 시간을 나타내는 정수형 데이터
2. return
    * time <= 30 이면 30 반환
    * time <= 60 이면 20 반환
    * time <= 90 이면 10 반환
    * time <= 120 이면 5 반환
    * time > 120 이면 1 반환

** Class : Map **
0. 활동 가능한 영역을 지정해준다.
1. attributes
    * target_location
        * safe_zone 의 중심점, 플레이어가 도달해야할 위치
        * 1차원 좌표를 나타내는 정수
        * event.py 모듈 내 get_random_location() 함수를 호출하여 초기화
2. methods
    * is_in_safe_zone(self, player, time)
        * target 위치와 플레이어 위치 사이의 거리를 계산
        * 계산된 거리가 get_safe_zone 함수에서 반환된 거리 이내인지를 True, False 로 반환
        * safe zone 밖의 플레이어의 health 수치를 시간당 5씩 감소

In [37]:
# from event import get_random_location

def get_safe_zone(time):
    if time <= 30:
        return 30
    elif time <= 60:
        return 20
    elif time <= 90:
        return 10
    elif time <= 120:
        return 5
    else:
        return 1
    
class Map:
    def __init__(self):
        self.target_location = get_random_location()
        
    def is_in_safe_zone(self, player, time):
        safe = abs(self.target_location - player.location) <= get_safe_zone(time)
        if not safe:
            player.health -= 10
        return safe
    
if __name__ == "__main__":
    for i in range(0,150,10):
        print(get_safe_zone(i))
    p = Player(1)
    m = Map()
    print(m.is_in_safe_zone(p,10))

30
30
30
30
20
20
20
10
10
10
5
5
5
1
1
False


## game.py
---
** Class : Game **
0. 게임을 구현하는 프로그램
1. attributes
    * num_player
        * 게임에 참여하는 플레이어 수
        * 생성자 함수로 입력 받음
    * players
        * Player 클래스의 객체를 아이템으로 포함하는 리스트
        * num_player 만큼 Player 객체를 생성하여 저장
    * map
        * Map 클래스의 객체
    * global_time
        * 0으로 초기화
2. methods
    * time_goes_by(self)
        * global_time 을 1 증가
        * 모든 플레이어 객체가 target_location 방향으로 한 칸 run 함
        * 플레이어 객체를 출력
        * 플레이어의 위치가 safe zone 내부에 있으면 'is in safe zone.'을 출력
        * game 객체 자체를 출력
    * `__str__`(self)
        * global_time 과 target_location 이 나타나도록 오버라이드

In [42]:
# from player import Player
# from map import Map

class Game:
    def __init__(self, num_player):
        self.num_player = num_player
        self.players = []
        for i in range(1,num_player+1):
            self.players.append(Player(i))
            
        self.map = Map()
        self.global_time = 0
        
    def __str__(self):
        game_inform = "Time = {} & Target Location = {}".format(self.global_time, self.map.target_location)
        return game_inform
    
    def time_goes_by(self):
        self.global_time += 1
        
        for p in self.players:
            if p.status == 'alive':
                p.run(self.map.target_location)
                self.map.is_in_safe_zone(p, self.global_time)
            
            p.status_update()
            print(p)
        
        for i in range(len(self.players)):
            if self.players[i].status == 'alive':
                players_to_fight = [self.players[i]]
                for j in range(len(self.players)):
                    if j == i:
                        continue
                        
                    if self.players[j].status == 'alive' and self.players[j].location == self.players[i].location:
                        players_to_fight.append(self.players[j])
                
                if len(players_to_fight) > 1:
                    battle(players_to_fight)
                
                for p in players_to_fight:
                    p.status_update()
        
        print(self)
        print("="*30)
        
if __name__ == "__main__":
    g = Game(20)
    while True:
        g.time_goes_by()
        alive_players = [player for player in g.players if player.status == 'alive']
        if len(alive_players) <= 1:
            
            break

Player 1 is alive. health = 100 location = 57
Player 2 is alive. health = 90 location = 6
Player 3 is alive. health = 100 location = 71
Player 4 is alive. health = 90 location = 35
Player 5 is alive. health = 100 location = 75
Player 6 is alive. health = 90 location = 1
Player 7 is alive. health = 100 location = 51
Player 8 is alive. health = 100 location = 95
Player 9 is alive. health = 100 location = 50
Player 10 is alive. health = 100 location = 57
Player 11 is alive. health = 90 location = 12
Player 12 is alive. health = 90 location = 20
Player 13 is alive. health = 90 location = 7
Player 14 is alive. health = 100 location = 77
Player 15 is alive. health = 90 location = 31
Player 16 is alive. health = 90 location = 35
Player 17 is alive. health = 100 location = 83
Player 18 is alive. health = 90 location = 24
Player 19 is alive. health = 90 location = 21
Player 20 is alive. health = 100 location = 67
battles between players!
	 Player 1 wins!
	 Player 10 loses...
battles between pla

In [43]:
import matplotlib.pyplot as plt

x = [i for i in range(50)]
y = [i**2 for i in range(50)]

for i in range(50):
    plt.plot(x[i], y[i])